# 1. Synthetic Data Generation

## 1.1 Imports

In [68]:
import numpy as np
from faker import Faker
import pandas as pd
import random
import random
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import nltk
from random import choice

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/linaismail/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

### Synthetic data

In [90]:
COLUMNS = ['Location', 'Patient Name', 'Study Date', 'DOB', 'Gender', 'Clinical History', 'Examination', 'Technical', 'Comparison', 'Findings', 'Anonimized']

data_no_identifiers = {'Location': ['167 Westmoreland Avenue SPRINGFIELD IL 62704', 
                               '425 East State Street ROCHESTER NY 14610',
                               '930 Franklin Street SEATTLE WA 98104',
                               '215 Maple Drive AUSTIN TX 78701',
                               '828 River Road NASHVILLE TN 37209'],
                       'Patient Name': ['THOMAS, GREGORY', 
                                        'EVANS, MICHAEL', 
                                        'CARTER, LUCAS',
                                        'ROBINSON, KEITH',
                                        'HARRIS, OWEN'],
                       'Study Date': ['03/22/2024', 
                                      '10/09/2024', 
                                      '06/04/2024',
                                      '12/15/2024',
                                      '11/08/2024'],
                       'DOB': ['05/18/1959', 
                               '09/03/1954', 
                               '08/15/1960',
                               '03/29/1965',
                               '02/16/1963'],
                       'Gender': ['Male', 
                                  'Male',
                                  'Male',
                                  'Male',
                                  'Male'],
                       'Clinical History': ['PROSTATE CA', 
                                            'SUSPECTED PROSTATE CA',
                                            'ELEVATED PSA, PROSTATE CA SCREENING',
                                            'PROSTATE CANCER FOLLOW-UP',
                                            'SCREENING FOR PROSTATE CANCER'],
                       'Examination': ['72195 MRI Pelvis W/O Contrast 02548697', 
                                       '72196 MRI Pelvis With and Without Contrast 03458724',
                                       '74175 CT Pelvis With Contrast 04598321',
                                       '76322 MRI Prostate With Dynamic Contrast Enhancement 05638472',
                                       '72195 MRI Prostate With and Without Contrast 07659248'],
                       'Technical': ['MRI examination of the prostate was performed on the GE Discovery MR750 3.0T high definition MRI magnet. Prostatic perfusion kinetic analysis is performed on an independent workstation with prostate volume calculations. 0.8 mg of Glucagon was administered intravenously.',
                                     'MRI examination of the prostate was conducted on the Siemens Magnetom Verio 3.0T MRI system. Dynamic contrast-enhanced (DCE) MRI and spectroscopy were performed. 0.2 mg of Atropine was administered intravenously for motion artifact reduction.',
                                     'CT examination of the pelvis was performed on the Toshiba Aquilion ONE 320-slice CT scanner. Pre-contrast and post-contrast images were acquired. 75 mL of iodinated contrast was administered intravenously.',
                                     'MRI examination of the prostate was conducted using the Hitachi Oasis 1.2T high-field open MRI system. Dynamic contrast-enhanced (DCE) MRI sequences were performed. An intravenous dose of 0.1 mg of Gadobutrol was used for contrast enhancement.',
                                     'MRI of the prostate was performed using the Siemens MAGNETOM Skyra 3T MRI system. Pre-contrast, dynamic contrast-enhanced (DCE), and diffusion-weighted imaging (DWI) sequences were acquired. A total of 0.1 mmol/kg body weight of Gadobutrol was used for contrast enhancement.'],
                       'Comparison': ['No relevant comparison.', 
                                      'No relevant comparison.',
                                      'No previous imaging available for comparison.',
                                      'Previous MRI dated 05/10/2024.',
                                      'No prior prostate MRI for comparison.'],
                       'Findings': ['MRI examination of the prostate was performed without IV gadolinium enhancement. The patient is not on any form of dialysis.\
                                    Serum PSA: 9.43 ng/ml, on 02/18/2024 \
                                    No post-surgical changes noted as the patient has not undergone prostatectomy. A heterogeneous 2.0 x 3.2 x 2.1 cm mass is observed in the right peripheral zone of the prostate. Abnormal diffusion and abnormal apparent diffusion coefficiency (ADC) are noted at the lesion, suggestive of malignancy.\
                                    No evidence of lymph node or bony metastasis is noted. No additional significant findings such as large fluid collections or hernias are observed.\
                                    Axial T1-weighted images of the abdomen and pelvis show no evidence of distant disease.', 
                                    'The MRI examination of the prostate utilized IV gadolinium contrast. No prior surgeries related to the prostate.\
                                    Serum PSA: 12.67 ng/ml, on 09/20/2024\
                                    A discrete 1.5 x 2.0 x 1.8 cm nodule is identified in the left anterior fibromuscular stroma of the prostate. The nodule demonstrates early contrast uptake and restricted diffusion, raising concern for malignancy.\
                                    There\'s no evidence of extracapsular extension. No lymphadenopathy or evidence of bone metastasis is apparent. The remainder of the pelvis is unremarkable without significant lymphadenopathy or ascites.',
                                    'CT examination of the pelvis shows a well-circumscribed 1.8 x 2.5 x 2.0 cm mass within the central zone of the prostate. The mass enhances post-contrast, indicative of vascularity, suggesting a neoplastic process.\
                                    Serum PSA: 15.22 ng/ml, measured on 05/28/2024\
                                    No evidence of extraprostatic extension. No lymphadenopathy or distant metastasis to the bones or organs is detected. The bladder and adjacent structures are unremarkable.',
                                    'The MRI examination of the prostate revealed a focal area of hypointensity on T2-weighted images measuring 2.2 x 3.0 x 2.5 cm in the posterior aspect of the right peripheral zone. The lesion shows significant enhancement post-contrast administration and exhibits restricted diffusion, which is consistent with the known diagnosis of prostate cancer.\
                                    Serum PSA levels have decreased to 8.34 ng/ml from 10.56 ng/ml on 05/01/2024, likely indicative of a positive response to treatment.\
                                    No new areas of abnormal enhancement or diffusion restriction are detected within the prostate. There is no evidence of extraprostatic extension or seminal vesicle invasion. No lymph node enlargement or bone metastases are evident on the images.',
                                    'The MRI scan reveals a 2.6 x 3.0 x 2.8 cm irregular mass in the peripheral zone of the left lobe of the prostate. The mass shows hypointense signal on T2-weighted images, and high signal intensity on DWI with corresponding low ADC values, suggesting restricted diffusion. There is marked enhancement on DCE-MRI, characteristic of prostate carcinoma.\
                                    Serum PSA: 20.35 ng/ml, measured on 10/29/2024\
                                    There\'s no evidence of extracapsular extension. No lymph node enlargement or bone metastasis is detected. The rest of the pelvis is unremarkable with no signs of secondary involvement.'],
                        'Impressions': ['1.	Heterogeneous 2.0 x 3.2 x 2.1 cm mass in the right peripheral zone of the prostate, suggestive of malignancy.\
                                        2.	No evidence of nodal or bony metastasis is noted.',
                                        '1.	Discrete 1.5 x 2.0 x 1.8 cm nodule in the left anterior fibromuscular stroma of the prostate with imaging features concerning for malignancy.\
                                        2.	No evidence of extracapsular extension, lymphadenopathy, or bony metastasis.',
                                        '1.	Well-circumscribed 1.8 x 2.5 x 2.0 cm enhancing mass within the central zone of the prostate, suspicious for prostate carcinoma.\
                                        2.	No evidence of extraprostatic extension or distant metastasis.',
                                        '1.	Persistent focal lesion in the posterior aspect of the right peripheral zone, consistent with the known prostate cancer, showing signs of treatment response.\
                                        2.	No evidence of disease progression or new metastatic disease.',
                                        '1.	A 2.6 x 3.0 x 2.8 cm mass in the peripheral zone of the left lobe of the prostate, highly suggestive of prostate carcinoma.\
                                        2.	No evidence of extracapsular extension, lymph node enlargement, or distant metastasis.'],
                        'Anonimized': ['True', 
                                       'True',
                                       'True',
                                       'True',
                                       'True']}

data_with_identifiers = {'Location': ['942 Valley View Drive PORTLAND OR 97205',
                                 '306 Cedar Lane MIAMI FL 33129',
                                 '519 Oakwood Avenue NEW YORK, NY 10023',
                                 '287 Pine Street CHICAGO, IL 60607',
                                 '634 Spruce Drive PHILADELPHIA, PA 19106'],
                         'Patient Name': ['JENKINS, TYLER', 
                                          'MARTINEZ, ALEX',
                                          'WILSON, BRIAN',
                                          'DIAZ, MARCO',
                                          'ROBERTS, NATHAN'],
                         'Study Date': ['07/13/2024',
                                        '02/27/2024',
                                        '05/19/2024',
                                        '08/03/2024',
                                        '04/22/2024'],
                         'DOB': ['07/24/1968',
                                 '12/10/1970',
                                 '04/08/1974',
                                 '09/15/1969',
                                 '06/11/1972'],
                         'Gender': ['Male',
                                    'Male',
                                    'Male',
                                    'Male',
                                    'Male'],
                         'Referring': ['PETERS, NP, CAROLYN',
                                       'EVANS, NP, LINDA',
                                       'REED, NP, SARAH',
                                       'HUGHES, NP, OLIVIA',
                                       'JENNINGS, NP, ALEX'],
                         'Acc': ['MORRIS, KYLE, NP; WANG, ELIZABETH, MD',
                                 'THOMPSON, JANE, NP; LIM, RICHARD, MD',
                                 'DAVIS, MICHELLE, NP; KIM, JONATHAN, MD',
                                 'GREEN, PATRICIA, NP; ALI, AHMED, MD',
                                 'MILLER, KEVIN, NP; YOUNG, SUSAN, MD'],
                         'Clinical History': ['EVALUATION OF ELEVATED PSA',
                                              'PROSTATE CANCER MONITORING',
                                              'POST-TREATMENT EVALUATION FOR PROSTATE CANCER',
                                              'HIGH-RISK PROSTATE CANCER SCREENING',
                                              'POST-RADIOTHERAPY ASSESSMENT FOR PROSTATE CANCER'],
                         'Examination': ['72195 MRI Prostate With Dynamic Contrast Enhancement 08459731',
                                         '72195 MRI Prostate Without Contrast 09367245',
                                         '72195 MRI Prostate With Dynamic Contrast Enhancement 10248756',
                                         '72195 MRI Prostate With Dynamic Contrast Enhancement 11348762',
                                         '72195 MRI Prostate With Dynamic Contrast Enhancement 12194760'],
                         'Technical': ['MRI of the prostate was performed using the Philips Achieva 3T MRI system. Sequences included T2-weighted images in three planes, diffusion-weighted imaging (DWI), and dynamic contrast-enhanced (DCE) MRI. Gadolinium-based contrast agent was administered at a dose of 0.2 mmol/kg.',
                                       'MRI of the prostate was conducted on the GE Signa Premier 3T. The exam included T2-weighted, diffusion-weighted imaging (DWI), and apparent diffusion coefficient (ADC) mapping sequences. No contrast was used in this study.',
                                       'The examination was performed on a Canon Vantage Galan 3T MRI system, including T2-weighted imaging, dynamic contrast-enhanced sequences, and diffusion-weighted imaging with ADC mapping. Intravenous gadolinium was administered at a standard dose.',
                                       'Performed on a Hitachi Echelon Oval 1.5T MRI system, including T2-weighted images in multiple planes, dynamic contrast-enhanced (DCE) sequences, and diffusion-weighted imaging (DWI) with apparent diffusion coefficient (ADC) mapping. Gadolinium contrast was administered.',
                                       'This MRI was performed on a Siemens MAGNETOM Verio 3T system. It included T2-weighted, DWI, and DCE sequences. Gadolinium-based contrast was administered at a standard dosage.'],
                         'Comparison': ['Comparison made with MRI dated 04/16/2024.',
                                        'Prior MRI on 11/30/2023 used for comparison.',
                                        'Comparison is made to MRI dated 01/20/2024.',
                                        'Previous MRI on 03/15/2024 for comparison.',
                                        'Compared to MRI dated 01/30/2024.'],
                         'Findings': ['A concerning lesion measuring 2.8 x 3.5 x 3.0 cm is observed in the anterior fibromuscular stroma of the prostate, [PATIENT_NAME] 07/24/1968 exhibits a hypointense signal on T2-weighted images and pronounced enhancement post-contrast. The lesion demonstrates restricted diffusion on DWI, and the ADC map shows low ADC values, suggestive of malignancy. No sign of extracapsular extension is detected. The serum PSA level, as of 07/01/2024, was 22.48 ng/ml, indicating a possible progression of the pathology in [PATIENT_NAME]. No lymphadenopathy or signs of distant metastasis are observed. The bladder and seminal vesicles appear normal.',
                                      'The MRI reveals a lesion in the right peripheral zone, measuring 1.9 x 2.2 x 2.0 cm, with characteristics suggestive of prostate cancer. The lesion, noted in the record of patient [PATIENT_NAME] (D.O.B. 12/10/1970), shows a markedly hypointense signal on T2-weighted images and increased signal intensity on the DWI sequences. The corresponding ADC map highlights low ADC values, typical for malignant lesions. Serum PSA was reported at 17.32 ng/ml on 02/15/2024, an increase from previous levels noted in [PATIENT_NAME]\'s records. There is no evidence of extraprostatic extension or seminal vesicle involvement. The lymph nodes and adjacent structures are within normal limits, with no signs of metastasis identified.',
                                      'There is a residual lesion measuring approximately 1.4 x 1.8 x 1.6 cm located in the left peripheral zone, [PATIENT_NAME], indicative of persistent disease. This area demonstrates [ADDRESS] characteristics with hypointensity on T2-weighted images, enhanced signal on DCE sequences, and restricted diffusion on DWI. The lesion\'s imaging features are consistent with those of residual or recurrent prostate cancer. The PSA level as of 05/01/2024 was 11.76 ng/ml, a slight decrease from previous values. No lymph node enlargement or evidence of metastatic disease is present. The bladder and seminal vesicles are unremarkable.',
                                      'A lesion identified in the posterior segment of the right peripheral zone, measuring 2.2 x 2.7 x 2.4 cm, [PATIENT_NAME] 09/15/1969 shows significant contrast enhancement and restricted diffusion, characteristic of malignancy. The lesion\'s ADC value is markedly low, correlating with high cellularity. Notably, 287 Pine Street, the serum PSA level reported on 07/28/2024 was 19.82 ng/ml, up from 15.67 ng/ml at the time of the last MRI. There is no clear evidence of extracapsular extension or seminal vesicle invasion. The rest of the prostate appears homogeneous. No pelvic lymphadenopathy or distant metastasis is detected.',
                                      'The MRI scan identified a post-treatment residual lesion in the prostate\'s left hemisphere, measuring 1.6 x 2.1 x 1.8 cm. This lesion, associated with patient [PATIENT_NAME]] (D.O.B. 06/11/1972), [ADDRESS], demonstrated characteristics suggestive of persistent cancerous tissue, including hypointensity on T2-weighted sequences and elevated signal intensity on DCE-MRI. The DWI sequence, alongside the ADC map, indicated restricted diffusion within the lesion. The serum PSA level as of 04/15/2024 was 13.45 ng/ml, reflecting a decrease from prior measurements. No evidence of metastasis to lymph nodes or distant organs was observed. The bladder and seminal vesicles appeared normal, with no signs of invasion.'],
                         'Impressions': ['1.	A 2.8 x 3.5 x 3.0 cm lesion in the anterior fibromuscular stroma of the prostate, with imaging features highly suggestive of malignancy in patient JENKINS, TYLER, D.O.B. 07/24/1968.\
                                         2.	No evidence of extracapsular extension or distant metastasis.',
                                         '1.	A 1.9 x 2.2 x 2.0 cm lesion in the right peripheral zone of the prostate, suggestive of malignancy in the context of ALEX MARTINEZ\'s clinical background.\
                                        2.	No evidence of extracapsular spread or distant metastasis.',
                                        '1.	Residual lesion in the left peripheral zone of the prostate, measuring 1.4 x 1.8 x 1.6 cm, suggestive of persistent or recurrent prostate cancer.\
                                        2.	No evidence of metastatic spread or lymph node involvement.',
                                        '1.	Significant lesion in the right peripheral zone, suggestive of prostate cancer, in patient DIAZ, MARCO, with an address at 287 Pine Street and D.O.B. of 09/15/1969.\
                                        2.	Stable disease with no evidence of progression or metastasis.',
                                        '1.	Persistent lesion in the prostate\'s left hemisphere, indicative of residual or recurrent prostate cancer in NATHAN ROBERTS, residing at 634 Spruce Drive.\
                                        2.	No metastasis or new areas of concern post-radiotherapy.'],
                         'Anonimized': ['False',
                                        'False',
                                        'False',
                                        'False',
                                        'False']}

df_no_identifiers = pd.DataFrame(data_no_identifiers)
df_with_identifiers = pd.DataFrame(data_with_identifiers)

combined_df = pd.concat([df_no_identifiers, df_with_identifiers], ignore_index=True)

#### Generate more findings with identifiers

In [82]:
def generate_new_findings(existing_findings, num_new_points=10):
    tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
    model = GPT2LMHeadModel.from_pretrained("gpt2")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    random.seed(42)

    new_findings = []
    for i in range(num_new_points):
        existing_finding = random.choice(existing_findings)
        input_ids = tokenizer.encode(existing_finding, return_tensors="pt").to(device)

        sample_output = model.generate(
            input_ids,
            max_length=len(input_ids[0]) + 50,
            temperature=0.7,
            num_return_sequences=1,
            pad_token_id=tokenizer.eos_token_id,
            top_p=0.9,
            do_sample=True
        )

        generated_text = tokenizer.decode(sample_output[0], skip_special_tokens=True)

        new_findings.append(generated_text)
    
    return new_findings


In [91]:
data_generated_no_id = generate_new_findings(data_no_identifiers['Findings'], 10)
data_generated_with_id = generate_new_findings(data_with_identifiers['Findings'], 10)

In [96]:
#Remove newlines 
for i in range(10):
    data_generated_no_id[i] = data_generated_no_id[i].replace('\n', ' ')
    data_generated_with_id[i] = data_generated_with_id[i].replace('\n', ' ')

print('No identifiers')
for i in range(10):
    print(data_generated_no_id[i])

print('With identifiers')
for i in range(10):
    print(data_generated_with_id[i])


No identifiers
MRI examination of the prostate was performed without IV gadolinium enhancement. The patient is not on any form of dialysis.                                    Serum PSA: 9.43 ng/ml, on 02/18/2024                                     No post-surgical changes noted as the patient has not undergone prostatectomy. A heterogeneous 2.0 x 3.2 x 2.1 cm mass is observed in the right peripheral zone of the prostate. Abnormal diffusion and abnormal apparent diffusion coefficiency (ADC) are noted at the lesion, suggestive of malignancy.                                    No evidence of lymph node or bony metastasis is noted. No additional significant findings such as large fluid collections or hernias are observed.                                    Axial T1-weighted images of the abdomen and pelvis show no evidence of distant disease.                                                  
MRI examination of the prostate was performed without IV gadolinium enhancement. The patient is not

## 1.2 Generate Data

In [93]:
COLUMNS = ['Patient ID', 'Name', 'Ethnicity', 'Age', 'Location', 'Total PSA (ng/ml)', 'Clinically Significant', 'PI-RAD', 'MRI Date', 'Tumor Marker Results', 'Conclusions', 'Anonimized']
ETHNICITIES = ["White", "Black", "Hispanic", "Asian", "Native American", "Mixed", "Other"]

fake = Faker()
fake.locale = 'en_US'
data = {col: [] for col in COLUMNS}

ages = np.random.normal(50, 15, size=10000)
ages = np.clip(ages, 20, 90)
data['Age'] = [int(age) for age in ages]


for i in range(10000):
    if random.random() < 0.3: 
        data['Patient ID'].append('' if random.random() < 0.6 else 'PRO' + str(i).zfill(8))
        data['Name'].append('' if random.random() < 0.6 else fake.first_name_male() + ' ' + fake.last_name())
        data['Ethnicity'].append('' if random.random() < 0.6 else random.choice(ETHNICITIES))
        data['Location'].append('' if random.random() < 0.6 else fake.address().replace('\n', ', '))
        if data['Patient ID'][-1] == '' and data['Name'][-1] == '' and data['Ethnicity'][-1] == '' and data['Location'][-1] == '':
            data['Anonimized'].append('Yes')
        else:
            data['Anonimized'].append('No')
    else:
        data['Patient ID'].append('PRO' + str(i).zfill(8))
        data['Name'].append(fake.first_name_male() + ' ' + fake.last_name()) 
        data['Ethnicity'].append(random.choice(ETHNICITIES))
        data['Location'].append(fake.address().replace('\n', ', '))
        data['Anonimized'].append('No')

    data['MRI Date'].append(fake.date_between(start_date='-1y', end_date='today'))

    total_psa = round(random.uniform(0, 24), 2) 
    clinically_significant = 'No'
    pi_rad = random.randint(1, 2)
    tumor_marker_results = 'Low'
    conclusions = 'WE SUGGEST TO DISCARD MALIGNANCY.'

    if total_psa > 4:
        clinically_significant = 'Yes'
        pi_rad = random.randint(3, 5)
    
    if pi_rad > 2:
        tumor_marker_results = random.choice(['Moderate', 'High'])
        
    if tumor_marker_results == 'High' or tumor_marker_results == 'Moderate':
        if data['Anonimized'][-1] == 'No' and data['Name'][-1] != '' and data['Location'][-1] != '':
            conclusions = data_generated_with_id[random.randint(0, 9)]
            conclusions = conclusions.replace('[PATIENT_NAME]', data['Name'][-1]).replace('[ADDRESS]', data['Location'][-1])
        else: 
            conclusions = data_generated_no_id[random.randint(0, 9)]

    data['Total PSA (ng/ml)'].append(total_psa)
    data['PI-RAD'].append(pi_rad)
    data['Tumor Marker Results'].append(tumor_marker_results)
    data['Clinically Significant'].append(clinically_significant)
    data['Conclusions'].append(conclusions)

## 1.3 Store in CSV File

In [94]:
df = pd.DataFrame(data)
df.to_csv('new_data.csv')